### Just a bit better

In the last section, we saw our process for improviding regression lines.  We started with a simple regression line of the form, $y = mx + b $ to predict an output given an input.  Then, we said that each difference between the output that our regression line predicts and the actual value an error.    

![regression-scatter.png](./regression-scatter.png)

We then found away to assign a quantity to how well our regression line mapped to our data by squaring all of the errors (to eliminate negative values) and then adding these squares, our residual sum of squares (RSS).  Armed with a number that describes "goodness of fit", we can then begin to iteratively try new regression lines by adjusting our y-intercept value, $b$, or slope value, $m$, and assessing goodness of fit.  By finding the values $m$ and $b$ that minimize the RSS, we can find our "best fit line".  In our cost function below, you can different values of RSS produced from $b$ values, with the bottom of curve displaying the $b$ value that produces the lowest RSS.

![](./cost-curve-plot.png)

In the last section, we decided that we don't want to try **all** of the different values for a regression line, $m$ and $b$.  Doing so takes too long, as there are too many possibilities. And as you'll see in the future, our regression formulas will only get more complicated.  This technique doesn't work.

So in the last lesson, we arbitrarily checked in increments of 10 to see if our y-intercept value is creating a higher or lower RSS.  That is our technique.  We make a change and if the change decreases our cost curve we keep moving y-intercept in that direction.  

Add in chart here:

### Still, things are not so simple

The only thing we want to differently is take our steps with a more assurance that we will be approaching our minimum RSS.  Adjusting the y-intercept value and then checking the effect is a bit like trying to fly plane by just moving sitting down and pressing buttons.  Is there a way that even with our first change, we can rest assured we're moving in the right direction?  And how do we know how large a change to make to our y-intercept.  

![](https://bossip.files.wordpress.com/2014/11/aden-and-cree-580x435.jpg)

We call each of these changes to our variables our step size.  Our task for this lesson is to take have step sizes that get us to our RSS quickly without overshooting the mark.

### Our technique: look to the slope

Let's take another look at our cost function.

![](./cost-curve-plot.png)

In fact, let's just look at one slice of it.  Looking at just one part of our cost function below, do you see a way that even without knowing the minimum value, we can still get a sense of the direction and size of our next step?  

**Add in slice of cost curve here **

You likely can if you imagine yourself standing on your cost curve.  Even with your eyes closed, you could tell simply by the way you were tilting whether the bottom of the curve were forwards or backwards.  And the steeper the tilt, the further away you are from the bottom and the larger the step you take.  So by looking to the slope of our cost curve, we can tell where our next step should be.  If the slope tilts downwards at that point  we walk forward, and if the slope tilts upwards at that point we walk backwards.  Then the steeper the tilt, the larger the step and the flatter the tilt the smaller the step we should take as we should be approaching our ideal y-intercept.  

So now, by looking to the tilt of a cost curve at a given point, we can discover the direction of our next step and how large of step to take.  

### Stepping according to the slope

In [1]:
import plotly
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

first_show = {'x': 100, 'y': 275}
second_show = {'x': 200, 'y': 300}
third_show = {'x': 400, 'y': 700}

shows = [first_show, second_show, third_show]

def y(x, points):
    point_at_x = list(filter(lambda point: point['x'] == x,points))[0]
    return point_at_x['y']

def squared_error(point, m, b):
    return (point['y'] - (m*point['x'] + b))**2

def rss(points, m, b):
    squared_errors = list(map(lambda point: squared_error(point, m, b), points))
    return sum(squared_errors)

def cost_chart_b(points, m, b_values):
    rss_values = list(map(lambda b: rss(points, m, b), b_values))
    return {'b_values': b_values, 'rss_values': rss_values}

b_values = [65, 70, 75, 80, 85, 90, 95]

cost_chart = cost_chart_b(shows, 1.417, b_values)


In [5]:
def cost_curve_at(b, m, points):
    delta = .01
    base_rss = rss(points, m, b)
    numerator = rss(points, m, b + delta) - base_rss
    slope = numerator/delta
    return {'b': b,'rss': base_rss, 'slope': slope}

def plot(traces):
    plotly.offline.iplot(traces)

def build_tangent_line(b, m, points, line_length = 5):
    curve_at_point = cost_curve_at(b, m, points)
    slope = curve_at_point['slope']
    b_minus = b - line_length
    b_plus = b + line_length
    rss_exact = curve_at_point['rss']
    rss_minus = rss_exact - slope * line_length
    rss_plus = rss_exact + slope * line_length
    return {'x': [b_minus, b, b_plus], 'y': [rss_minus, curve_at_point['rss'], rss_plus], 'mode': 'lines+text', 'text': ['    slope:' + format(slope, '.2f')], 'textposition': 'right'}

m = cost_curve_at(100, 1.417, shows)
tangent_lines = list(map(lambda b_value: build_tangent_line(b_value, 1.417, shows, 2), [70, 84.61, 90.85]))
tangent_lines
cost_trace = {'x': cost_chart['b_values'], 'y': cost_chart['rss_values'], 'mode':'line'}
plot([cost_trace, *tangent_lines])

The slope of curve at any given point is equal to the slope of the tangent line at that point.  A tangent line is the line that just barely touches the curve at that point.  In the above graph, the orange, green, and red lines are tangent to our cost curve at the points where $b$ equals 70, 84, and 90 respectively.  

Now just look at the slope of the three tangent lines which are labeled above them: -146, -58, -21.  (We'll talk about how to calculate the slope of a curve at a given point in future lessons.)You can see that even without knowing the value of $b$ that produces the minimum RSS, using the slope of the tangent line to indicate the size of and direction of our next guess at $b$ points us in the right direction.  

For example, let's use the following formula for approaching our $b$ finding the ideal $b$: (1) randomly choose a value of $b$, and then (2)  for every next value of $b$, use the formula $ b = (-.1) * slope_{b = i} + b_i$.  It looks like this: 

* $b_{t=0} = 70 $
* $b_{t=1} = (-.1) * -146.17  + 70 = 14.61 + 70 = 84.61 $
* $b_{t=2} = (-.1) * -58.51 + 85 = 5.851 + 85 = 90.851 $
* $b_{t=3} = (-.1) * -21.07 + 90.85 = 90.851 + 2.107 $

First notice that we multiply our slope of our tangent line by a negative number to determine how to update our value of $b$.  We do so because when the slope of our tangent line points downwards, meaning it's negative, we should increase our $b$ value to decrease our RSS.    

Also notice that we don't update our values of $b$ by just adding or subtracting the slope.  Doing so would be too drastic.  Instead we multiply by the slope by a fraction -- in this case -- $.1$ which is called a **learning rate**.  This way, the steeper slope of the tangent line, the more we change in $b$, but we still make sure we are not changing our regression lines too drastically.  
> We should point out that, in general, learning rates are much smaller than what we saw here (.0001 is more typical), but we used this learning rate just for demonstration purposes.   

Finally, this technique is pretty magical.  By looking at the tangent line at each point, we no longer are just changing our $b$ value and hoping that it has the correct impact on our RSS.  This is because, for one, the slope of the tangent line points us in the right direction.  And as you can see above, our technique properly adjusts the amount to change the $b$ value by without even knowing where ideal $b$ value is.  When our $b$ was far away from the ideal $b$ value our formula had us increase our $b$ by 14, and in just three steps we were only updating our $b$ value by 2, as we approached the $b$ that minimizes our RSS.  

### Summary

We started this section with saying that we wanted a technique to find a $b$ value that would minimize our RSS, given a value of $m$.  We did not want to simply try all of the values of $b$ or $m$ as doing so would be inefficient.  Instead, we went with the approach of gradient descent discussed in previous lessons, where we try variations of regression lines by iteratively changing our $b$ variable and assessing our RSS to see if we are making progress.

In this lesson, we focused in on how to know which direction to alter a given variable, $m$ or $b$, as well as a technique for changing the magnitude of our change to one of these variables.  We said that by using the line tangent to the cost curve at a given point to indicate the direction and size of the update to $b$, we can approach our $b$ value that minimizes RSS efficiently.  The further away, the steeper the curve and thus the larger the step we would want to take.  Appropriately, our tangent line slope would have us take a larger step.  And the closer we are to the ideal $b$ value, the flatter the tangent line to the curve, and the smaller a step we would take. 